# Imports 

In [1]:
import sys
sys.path.append('/home/jovyan/work/sem-covid/')
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')

import spacy
import pytextrank

from tqdm.notebook import tqdm
from sem_covid.services.data_registry import Dataset
from transformers import AutoTokenizer, AutoModel
from summarizer import Summarizer

2021-08-22 10:32:22.509265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-22 10:32:22.509286: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('textrank')

# Methods

In [37]:
def extract_most_important_sentences_pytextrank(paragraph: str, top_n=5) -> list:
    
    top_n_most_important_sentences = []
    
    doc = nlp(paragraph)

    for sent in doc._.textrank.summary(limit_phrases=15, limit_sentences=top_n):
        top_n_most_important_sentences.append(str(sent))

    return top_n_most_important_sentences

# Fetch the data

In [4]:
pwdb = Dataset.PWDB.fetch()[['background_info_description', 'content_of_measure_description']]

100% (1288 of 1288) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [5]:
pwdb.head()

,background_info_description,content_of_measure_description
_id,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,"During the COVID-19 crisis, teleworking has be...",Applied for a period of three years and coveri...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,"In the face of the Corona crisis, employers ha...",The COVID-19 risk group regulation lists the m...
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,The Council of Ministers has agreed to authori...,"In total, €136.3 million will be allocated to ..."
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,"The Act 136/2020 Coll., selected modifications...",Eligible are all self-employed persons perform...
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,After the Financial Shield 1.0 was impelemente...,The main characteristics of the measure are as...


# Main

1. PyTextRank

In [13]:
for p in tqdm(pwdb['background_info_description'][:10]):
    print(extract_most_important_sentences_pytextrank(p))
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

['Discussions between the two representative social partners OGBL and LCGB and the employer association UEL, together with the Ministry of Work and Employment, led in a first instance to a joint assessment on teleworking at the level of the Economic and Social Council CES.', 'During the COVID-19 crisis, teleworking has been identified as a vital pillar to keep companies working and prevent social hardship.', 'From there, discussions continued between the social partners and an inter-professional agreement could have been signed between the social partners in October 2020.']

['Depending on the activity, this includes, for example, the possibilities for home office, redesigning workplaces to maintain a safe distance, installed barriers such as plexiglass walls or the use of personal protective equipment.', 'Employed workers with a very high risk of developing a serious illness if becoming infected with the Corona virus are entitled to work from their home office or to a change in workin

In [14]:
for p in tqdm(pwdb['content_of_measure_description'][:10]):
    print(extract_most_important_sentences_pytextrank(p))
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

["\r* Teleworking is based on a written agreement between the employer and the employee, containing several compulsory elements, such as for example the location where telework takes place and the number of hours, and an employee can't be dismissed if he/she does not accept a teleworking scheme and an employer can't be made accountable if the employer refuses a teleworking scheme.", '\r\r* Teleworking is identified as a form of organisational work, conducted by digital means and usually done at the company, but that is transferred to the location where the employee lives.', 'Applied for a period of three years and covering all sectors in Luxembourg (with the exception of transport), the agreement provides a definition of teleworking:', '\r* The employer must also provide to the employee all the technical equipment to efficiently work under a teleworking scheme.', '\r* The company staff delegation is regularly informed about the teleworking schemes.']

['active cancer with oncological p

2. BERT

In [40]:
bert_base_uncased_eurlex_tokenizer = AutoTokenizer.from_pretrained('nlpaueb/bert-base-uncased-eurlex')
bert_base_uncased_eurlex_model = AutoModel.from_pretrained('nlpaueb/bert-base-uncased-eurlex', output_hidden_states=True)

Some weights of the model checkpoint at nlpaueb/bert-base-uncased-eurlex were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
bert_summarizer = Summarizer(custom_model=bert_base_uncased_eurlex_model, custom_tokenizer=bert_base_uncased_eurlex_tokenizer)

In [45]:
for p in tqdm(pwdb['background_info_description'][:10]):
    print(bert_summarizer(p, num_sentences=5))
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

During the COVID-19 crisis, teleworking has been identified as a vital pillar to keep companies working and prevent social hardship. Discussions between the two representative social partners OGBL and LCGB and the employer association UEL, together with the Ministry of Work and Employment, led in a first instance to a joint assessment on teleworking at the level of the Economic and Social Council CES. From there, discussions continued between the social partners and an inter-professional agreement could have been signed between the social partners in October 2020.

In the face of the Corona crisis, employers have taken numerous measures to protect employees from infections. Depending on the activity, this includes, for example, the possibilities for home office, redesigning workplaces to maintain a safe distance, installed barriers such as plexiglass walls or the use of personal protective equipment. The third COVID-19 Law paved the way for defining specific "risk groups", to which fur

In [43]:
for p in tqdm(pwdb['content_of_measure_description'][:10]):
    print(bert_summarizer(p, num_sentences=5))
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

* Teleworking is identified as a form of organisational work, conducted by digital means and usually done at the company, but that is transferred to the location where the employee lives. * The work is considered as teleworking when it is applied to occasional and exceptional circumstances, and when it remains below the 10% threshold of the annual working time. * Teleworking is based on a written agreement between the employer and the employee, containing several compulsory elements, such as for example the location where telework takes place and the number of hours, and an employee can't be dismissed if he/she does not accept a teleworking scheme and an employer can't be made accountable if the employer refuses a teleworking scheme. * The employer must also provide to the employee all the technical equipment to efficiently work under a teleworking scheme. * The company staff delegation is regularly informed about the teleworking schemes.

8. arterial hypertension with existing end org